In [ ]:
%pip install geopandas
%pip install rasterio
%pip install utm

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import geopandas as gpd
from shapely.geometry.polygon import Polygon
import timeit
import shutil
import rasterio
import os

from sentinel2_download import to_zones, get_all_zones, get_sentinel_composite_data, remove_from_sagemaker, list_s3_files

In [ ]:
# Set session and s3 connection objects
sess = sagemaker.Session()
role = get_execution_role()
conn = boto3.client("s3")
bucket = "w210-poverty-mapper"
s3_path = "sentinel2_composite/raw_data"

In [ ]:
# Load country shapes
country_shapes = gpd.read_file("s3://w210-poverty-mapper/shape_files/raw_data/ipums_world_shapes/world_countries_2020.shp")

In [ ]:
# Get unique UTM zones
bd_list = ["Bangladesh"]
np_list = ["Nepal"]
ph_list = ["Philippines"] # no 52Q in data
tj_list = ["Tajikistan"] 
tl_list = ["Timor Leste"]
full_country_list = ["Bangladesh", "Nepal", "Philippines", "Tajikistan", "Timor Leste"]

for country_list in [bd_list, np_list, ph_list, tj_list, tl_list]:
    zone_list = get_all_zones(country_list, country_shapes)
    print(country_list[0], zone_list)

full_zone_list = get_all_zones(full_country_list, country_shapes)
print("De-duplicated Full List", full_zone_list)

In [ ]:
# Get data for all country zones in utm dictionary
get_sentinel_composite_data(full_zone_list[:-1])

In [ ]:
# Move directories to S3
move_to_s3(full_zone_list[:-1])

In [ ]:
# Check files in s3
list_s3_files(bucket, s3_path)

In [ ]:
# Remove from sagemaker "root"
remove_from_sagemaker(full_zone_list[:-1])

In [ ]:
# Example - delete s3 directory
#s3 = boto3.resource("s3")
#s3_bucket = s3.Bucket("w210-poverty-mapper")
#for obj in s3_bucket.objects.filter(Prefix="directory_path"):
#    s3.Object(s3_bucket.name,obj.key).delete()